In [0]:
from pyspark.sql.functions import explode, col

df = (
    spark.read
    .option("multiLine", True)
    .json("/Workspace/Users/gloomy.penguin.kim@gmail.com/data-5035-2026/week03/negotiated_rates.json")
)

df_flat = (
    df 
    .withColumn("oon", explode("out_of_network"))
    .withColumn("allowed", explode("oon.allowed_amounts"))
    .withColumn("provider", explode("allowed.payments.providers"))
    .withColumn("npi", explode("provider.npi"))
    .select(
        col("oon.name").alias("service_name"),
        col("oon.billing_code_type"),
        col("oon.billing_code"),
        col("oon.description"),
        col("allowed.service_code"),
        col("allowed.billing_class"),
        col("allowed.payments.allowed_amount"),
        col("provider.billed_charge"),
        col("npi")
    )
)

df_flat.show(truncate=True)

import json 
f = json.load(open("/Workspace/Users/gloomy.penguin.kim@gmail.com/data-5035-2026/week03/negotiated_rates.json"))
output = [] 
for obj in f["out_of_network"]:
  for obj2 in obj["allowed_amounts"]:
    for obj3 in obj2["payments"]["providers"]:
      for obj4 in obj3["npi"]:
        j = { "name": obj["name"], 
            "billing_code_type": obj["billing_code_type"], 
            "billing_code": obj["billing_code"], 
            "description": obj["description"],
            "service_code": obj2["service_code"],
            "billing_class": obj2["billing_class"],
            "allowed_amount": obj2["payments"]["allowed_amount"],
            "billed_charge": obj3["billed_charge"],
            "npi": obj4 
            }
        output.append(j)    
